In [5]:
import tensorflow as tf
# import numpy
import random
import math

In [11]:
# tf Graph Input
num_inputs = 16
num_hidden_1 = num_inputs*32
num_hidden_2 = num_inputs*32
num_outputs = 16
stddev = 1.0 

with tf.device('/gpu:0'):
    inputs = tf.placeholder(dtype=tf.float32, shape=[None, num_inputs], name="inputs")
    expected = tf.placeholder(dtype=tf.float32, shape=[None, num_outputs], name="expected")

    # First hidden layer of the network 
    h1w = tf.Variable(expected_shape=[num_inputs, num_hidden_1], name="hidden_1_weight", 
                     initial_value=tf.truncated_normal([num_inputs, num_hidden_1], stddev=stddev))
    h1b = tf.Variable(expected_shape=[1, num_hidden_1], name="hidden_1_bias", 
                     initial_value=tf.truncated_normal([1, num_hidden_1], stddev=stddev))

    h1l = tf.nn.sigmoid(tf.add(tf.matmul(inputs, h1w), h1b))

    # Second hidden layer of the network 
    h2w = tf.Variable(expected_shape=[num_hidden_1, num_hidden_2], name="hidden_2_weight", 
                     initial_value=tf.truncated_normal([num_hidden_1, num_hidden_2], stddev=stddev))
    h2b = tf.Variable(expected_shape=[1, num_hidden_2], name="hidden_2_bias", 
                     initial_value=tf.truncated_normal([1, num_hidden_2], stddev=stddev))

    h2l = tf.nn.sigmoid(tf.add(tf.matmul(h1l, h2w), h2b))

    # Weight and biases used to turn hidden layer into final output value. 
    ow = tf.Variable(expected_shape=[num_hidden_2, num_outputs], name="output_weight", 
                     initial_value=tf.truncated_normal([num_hidden_2, num_outputs], stddev=stddev))
    ob = tf.Variable(expected_shape=[1, num_outputs], name="output_bias", 
                     initial_value=tf.truncated_normal([1, num_outputs], stddev=stddev))

    output = tf.add(tf.matmul(h2l, ow), ob)

    # We compute error as the mean square difference 
    error = tf.reduce_mean(tf.pow(tf.subtract(output, expected), 2))

In [12]:
# Parameters
learning_rate = 0.0025

# Gradient descent
# optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(error)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(error)

In [13]:
training_steps = 10000

min = 1
max = 2**num_inputs
parity_max = 16

display_step = 500
batch_size = 32

def binlist(x, length):
    return [1.0 * (0!=(x&(1<<b))) for b in range(length)]

def onehot(x, length):
    return [1.0 * (x==b) for b in range(length)]

def lesshot(x, length):
    return [1.0 * (x<=b) for b in range(length)]

def zero(length): return [0] * length 

def makeinput(x):
    return binlist(x, num_inputs)

def f(x):  
    binx = binlist(x, num_inputs)
    numbits = sum([binx[i] for i in range(parity_max)])
    return onehot(numbits, num_inputs)

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_steps):
        data = { inputs: [], expected: []}
        for b in range(batch_size):
            x = random.randint(min,max)
            data[inputs].append(makeinput(x))
            data[expected].append(f(x))
        sess.run(optimizer, feed_dict=data)

        if epoch % display_step == 0:
            e = sess.run(error, feed_dict=data)
            print "Epoch:", epoch, " error=", e
    for x in range(min, parity_max):
        data = {inputs: [makeinput(x)], expected: [f(x)]}
        print (" x=", x, " output=", sess.run(output, feed_dict=data), 
        " expected=", data[expected], 
        " err=",  sess.run(error, feed_dict=data))

Epoch: 0  error= 160.914
Epoch: 500  error= 0.174251
Epoch: 1000  error= 0.139204
Epoch: 1500  error= 0.138081
Epoch: 2000  error= 0.150861
Epoch: 2500  error= 0.139574
Epoch: 3000  error= 0.127498
Epoch: 3500  error= 0.140545
Epoch: 4000  error= 0.177505
Epoch: 4500  error= 0.160604
Epoch: 5000  error= 0.124091
Epoch: 5500  error= 0.145918
Epoch: 6000  error= 0.119354
Epoch: 6500  error= 0.12316
Epoch: 7000  error= 0.160334
Epoch: 7500  error= 0.171522
Epoch: 8000  error= 0.153141
Epoch: 8500  error= 0.165144
Epoch: 9000  error= 0.11152
Epoch: 9500  error= 0.114472
(' x=', 1, ' output=', array([[ 0.29957753,  0.3047058 ,  0.31570122,  0.48503971,  0.67529303,
         0.50044376,  0.67335773, -0.50412762,  0.04999673, -0.48846126,
        -0.39888275, -0.4102124 , -0.10306102,  0.3734566 ,  0.2925266 ,
         0.30002248]], dtype=float32), ' expected=', [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], ' err=', 0.20101775)
(' x=', 2, ' output=', arra

In [ ]:
# Regression result